During Low Volatility		Sell ATM Spreads and long OTM units that are Monthly Expiration
In low volatility, watch the term structure, sell spread naked first and if F1/F2 are closing, enter long units to neutralize delta of spread


During High Volatility  		Sell ITM Spreads (60 to 70 Deltas) and buy OTM units that are Weekly Expiration			
			
In high volatility, if no held positions, watch for term structure to backwardate and enter ITM spread + Units with weekly expiration					


In [1]:
# Importing necessary models
import warnings
warnings.filterwarnings('ignore')

import smtplib
import pandas as pd
import numpy as np
import datetime as dt
import pandas.stats.moments as st
import time
%matplotlib inline
from bs4 import BeautifulSoup as bs
import requests
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from sqlalchemy import *
from sqlalchemy import create_engine
import calendar
import sqlite3 as sql
pd.options.display.float_format = '{:,.4f}'.format

init_notebook_mode(connected=True)

import os
main_dir = os.getcwd()

dbs_dir = 'C:\\Users\\Fang\\Desktop\\Python Trading\\Trading\\Data\\DBs'

os.chdir('C:\\Users\\Fang\\Desktop\\Python Trading\\Trading\\Trading\\Modules\\DataCollection')

from yahoo_query import *

os.chdir('C:\\Users\\Fang\\Desktop\\Python Trading\\Trading\\Trading\\Modules\\Options')
from optionsFunctions import *

os.chdir('C:\\Users\\Fang\\Desktop\\Python Trading\\Trading\\Trading\\Modules\\Processing')
from hv_calc import *

spx_options_dir = 'D:\\Options Data\\SPX'

vix_options_dir = 'D:\\Options Data\\VIX'

os.chdir(spx_options_dir)

In [13]:
spx_close = yahoo_query('^GSPC',dt.datetime(2007,1,1))
spx_close.hist_prices_query()
spx_close = spx_close.hist_prices

vix = yahoo_query('^VIX',dt.datetime(2007,1,1))
vix.hist_prices_query()
vix = vix.hist_prices

vf = pd.read_csv('http://173.212.203.121/noko.csv', index_col = 0)

spx_close.index = pd.to_datetime(spx_close.index)
vix.index = pd.to_datetime(vix.index)
vf.index = pd.to_datetime(vf.index)
spots = pd.concat([spx_close[['^GSPC_adjclose']], vix[['^VIX_adjclose']], vf[['F1','F2']]],axis = 1).dropna()
spots.columns = ['SPX','VIX','F1','F2']


In [248]:
# Saving to Options Database
keep_columns = ['Strike','Expiry','Type','Symbol','Last','Bid','Ask','IV','Underlying_Price']
os.chdir(dbs_dir)
options_engine = create_engine('sqlite:///spxOptions.db', echo=False)
os.chdir(spx_options_dir)

for filename in os.listdir('.'):
    if filename.endswith(".csv"):
        curr_options = pd.read_csv(filename, index_col = 0).set_index('Quote_Time')[keep_columns]
        curr_options.index = pd.to_datetime(curr_options.index) - dt.timedelta(hours = 4)
        curr_options.index = curr_options.index.map(lambda x: x.replace(second=0))
        curr_options['Expiry'] = pd.to_datetime(curr_options.Expiry)
        curr_options['DTE'] = (curr_options.Expiry - curr_options.index).dt.days + 1
        curr_options['Mid'] = (curr_options['Ask'] + curr_options['Bid'])/2
        greeks = all_greeks(curr_options, interest_rate = 0.023869, q = 0, year = 252)
        greeks.index = pd.to_datetime(curr_options.index)
        puts = greeks[greeks.Type == 'put']
        calls = greeks[greeks.Type == 'call']
        puts.to_sql('puts', con=options_engine, if_exists='append', index_label = 'quoteTime')
        calls.to_sql('calls', con=options_engine, if_exists='append', index_label = 'quoteTime')
    else:
        continue

In [137]:
# Reading in options data at the close
keep_columns = ['Strike','Expiry','Type','Symbol','Last','Bid','Ask','IV','Underlying_Price']
failed = ['spx_2018-08-27-12-56','spx_2018-08-28-12-56','spx_2018-08-30-12-56',
          'spx_2018-10-02-12-56','spx_2018-10-03-12-56','spx_2018-10-05-12-56',
          'spx_2018-11-28-12-58','spx_2018-11-30-11-10']

puts = []
calls = []
failed_csvs = []

for filename in failed:
    curr_options = pd.read_csv(filename + '.csv', index_col = 0).set_index('Quote_Time')[keep_columns]
    curr_options.index = pd.to_datetime(curr_options.index).normalize()
    curr_options.Expiry = pd.to_datetime(curr_options.Expiry)
    curr_options = curr_options[curr_options.Expiry < dt.datetime(2019,3,1)]
    curr_options['DTE'] = (curr_options.Expiry - curr_options.index).dt.days + 1
    curr_options['Mid'] = (curr_options['Ask'] + curr_options['Bid'])/2
    puts.append(curr_options[curr_options.Type == 'put'])
    calls.append(curr_options[curr_options.Type == 'call'])

for data_date in pd.date_range(start='8/8/2018', end='12/7/2018'):
    filename = 'spx_{}-12-55.csv'.format(data_date.strftime('%Y-%m-%d'))
    try:
        curr_options = pd.read_csv(filename, index_col = 0).set_index('Quote_Time')[keep_columns]
    except:
        None
    else:
        curr_options.index = pd.to_datetime(curr_options.index).normalize()
        curr_options.Expiry = pd.to_datetime(curr_options.Expiry)
        curr_options = curr_options[curr_options.Expiry < dt.datetime(2019,3,1)]
        curr_options['DTE'] = (curr_options.Expiry - curr_options.index).dt.days + 1
        curr_options['Mid'] = (curr_options['Ask'] + curr_options['Bid'])/2
        puts.append(curr_options[curr_options.Type == 'put'])
        calls.append(curr_options[curr_options.Type == 'call'])
        
puts = pd.concat(puts,axis = 0).drop_duplicates()
calls = pd.concat(calls, axis = 0).drop_duplicates()

interest_rate = 0.023869
q = 0
year = 252

put_greeks = all_greeks(puts.sort_index(ascending = True))
put_greeks.index = pd.to_datetime(puts.sort_index(ascending = True).index)

call_greeks = all_greeks(calls.sort_index(ascending = True))
call_greeks.index = pd.to_datetime(calls.sort_index(ascending = True).index)

In [224]:
starting_date = dt.datetime(2018,11,7)

curr_puts = put_greeks[(put_greeks.DTE == 31) & 
                       (put_greeks.index == starting_date) &
                       (put_greeks.Delta >= -.4)].sort_values('Delta')[list(filter(lambda x: x not in ['Last',
                                                                                                      'Bid',
                                                                                                      'Ask'], put_greeks.columns.tolist()))]
curr_puts

,Strike,Expiry,Type,Symbol,IV,Underlying_Price,DTE,Mid,Rho,Vega,Gamma,Theta,Delta
Quote_Time,,,,,,,,,,,,,
2018-11-07,"2,785.0000",2018-12-07,put,SPXW181207P02785000,0.1494,"2,811.7200",31,36.1000,-1.4363,3.8096,0.0026,-0.5721,-0.3999
2018-11-07,"2,780.0000",2018-12-07,put,SPXW181207P02780000,0.1493,"2,811.7200",31,34.0500,-1.3879,3.7742,0.0026,-0.5679,-0.3866
2018-11-07,"2,775.0000",2018-12-07,put,SPXW181207P02775000,0.1522,"2,811.7200",31,32.9000,-1.3481,3.7409,0.0025,-0.5759,-0.3754
2018-11-07,"2,770.0000",2018-12-07,put,SPXW181207P02770000,0.1522,"2,811.7200",31,31.0500,-1.3018,3.6990,0.0025,-0.5711,-0.3627
2018-11-07,"2,765.0000",2018-12-07,put,SPXW181207P02765000,0.1547,"2,811.7200",31,29.9000,-1.2637,3.6606,0.0024,-0.5763,-0.3521
2018-11-07,"2,760.0000",2018-12-07,put,SPXW181207P02760000,0.1553,"2,811.7200",31,28.3000,-1.2210,3.6146,0.0024,-0.5726,-0.3403
2018-11-07,"2,755.0000",2018-12-07,put,SPXW181207P02755000,0.1577,"2,811.7200",31,27.3000,-1.1854,3.5729,0.0023,-0.5765,-0.3303
2018-11-07,"2,750.0000",2018-12-07,put,SPXW181207P02750000,0.1586,"2,811.7200",31,25.9500,-1.1457,3.5236,0.0023,-0.5730,-0.3193
2018-11-07,"2,745.0000",2018-12-07,put,SPXW181207P02745000,0.1592,"2,811.7200",31,24.6000,-1.1060,3.4712,0.0022,-0.5680,-0.3084


In [225]:

combo = ['SPXW181207P02745000','SPXW181207P02680000','SPXW181207P02415000']
print(put_greeks[put_greeks.Symbol.isin(combo) & 
                 (put_greeks.index == starting_date)])

short_leg = put_greeks[put_greeks.Symbol == combo[0]][['Mid','Vega','Gamma','Theta','Delta']]
long_leg = put_greeks[put_greeks.Symbol == combo[1]][['Mid','Vega','Gamma','Theta','Delta']]
unit = put_greeks[put_greeks.Symbol == combo[2]][['Mid','Vega','Gamma','Theta','Delta']]
combo_perf = long_leg - short_leg + unit*2
combo_perf[combo_perf.index >= starting_date].join(spots)

               Strike     Expiry Type               Symbol    Last     Bid  \
Quote_Time                                                                   
2018-11-07 2,415.0000 2018-12-07  put  SPXW181207P02415000  2.8000  2.0500   
2018-11-07 2,680.0000 2018-12-07  put  SPXW181207P02680000 16.6500 13.8000   
2018-11-07 2,745.0000 2018-12-07  put  SPXW181207P02745000 30.2300 24.4000   

               Ask     IV  Underlying_Price  DTE     Mid     Rho   Vega  \
Quote_Time                                                                
2018-11-07  2.2000 0.2726        2,811.7200   31  2.1250 -0.1732 0.9859   
2018-11-07 14.2000 0.1806        2,811.7200   31 14.0000 -0.7324 2.7958   
2018-11-07 24.8000 0.1592        2,811.7200   31 24.6000 -1.1060 3.4712   

            Gamma   Theta   Delta  
Quote_Time                         
2018-11-07 0.0004 -0.2918 -0.0481  
2018-11-07 0.0016 -0.5308 -0.2042  
2018-11-07 0.0022 -0.5680 -0.3084  


,Mid,Vega,Gamma,Theta,Delta,SPX,VIX,F1,F2
2018-11-07,-6.3500,1.2965,0.0001,-0.5465,0.0080,"2,813.8899",16.3600,16.8250,17.1250
2018-11-08,-6.5000,1.4616,0.0002,-0.6327,0.0022,"2,806.8301",16.7200,16.9750,17.1750
2018-11-09,-8.6000,1.4850,0.0002,-0.6482,0.0142,"2,781.0100",17.3600,17.4750,17.6750
2018-11-12,-13.5500,2.4512,0.0006,-1.0815,0.0115,"2,726.2200",20.4500,19.5750,18.7250
2018-11-13,-16.7500,2.4935,0.0009,-1.1060,-0.0189,"2,722.1799",20.0200,19.4250,18.9250
2018-11-14,-18.4000,2.7750,0.0008,-1.2423,0.0453,"2,701.5801",21.2500,20.1750,19.4250
2018-11-15,-17.9000,2.1327,0.0008,-1.0549,0.0116,"2,730.2000",19.9800,19.3750,19.1250
2018-11-16,-17.1500,1.5519,0.0004,-0.8541,0.0614,"2,736.2700",18.1400,18.2750,18.3250
2018-11-19,-27.2000,2.2714,0.0012,-1.2434,0.0515,"2,690.7300",20.1000,19.9250,19.3750
2018-11-20,-30.9500,3.4216,0.0023,-1.8293,-0.0026,"2,641.8899",22.4800,22.1750,20.4250


In [ ]:
combo = ['SPXW180907P02870000','SPXW180907P02850000','SPXW180907P02570000']
print(put_greeks[put_greeks.Symbol.isin(combo) & 
                 (put_greeks.index == dt.datetime(2018,8,8))])


combo = ['SPXW181031P02900000','SPXW181031P02875000','SPXW181031P02650000']
print(put_greeks[put_greeks.Symbol.isin(combo) & 
                 (put_greeks.index == dt.datetime(2018,10,1))])

combo = ['SPXW181029P02875000','SPXW181029P02850000','SPXW181029P02715000']
print(put_greeks[put_greeks.Symbol.isin(combo) & 
                 (put_greeks.index == dt.datetime(2018,10,17))])

In [10]:
options_lst = []
keep_columns = ['Strike','Expiry','Type','Symbol','Last','Bid','Ask','IV','Underlying_Price']

for filename in os.listdir('.'):
    if filename.endswith(".csv") and '2018-09-14' in filename:
        curr_options = pd.read_csv(filename, index_col = 0).set_index('Quote_Time')[keep_columns]
        curr_options.index = pd.to_datetime(curr_options.index) - dt.timedelta(hours = 4)
        curr_options.index = curr_options.index.map(lambda x: x.replace(second=0))
        curr_options['Expiry'] = pd.to_datetime(curr_options.Expiry)
        curr_options['DTE'] = (curr_options.Expiry - curr_options.index).dt.days + 1
        curr_options['Mid'] = (curr_options['Ask'] + curr_options['Bid'])/2
        
        options_lst.append(curr_options)
    else:
        continue

In [12]:
sept = pd.concat(options_lst, axis = 0)
sept[sept.DTE < 3].to_csv('sept-14-test.csv')

In [10]:
os.chdir(vix_options_dir)

vix_intraday_list = []

for filename  in os.listdir('.'):
    if filename.endswith('.csv'):
        curr_vix = pd.read_csv(filename, index_col = 0)[['Quote_Time','Underlying_Price']]
        curr_vix.Quote_Time = pd.to_datetime(curr_vix.Quote_Time) - dt.timedelta(hours = 4)
        vix_intraday_list.append(curr_vix.drop_duplicates())

In [24]:
vix_intraday = pd.concat(vix_intraday_list).reset_index(drop = True)
vix_intraday.Quote_Time = vix_intraday.Quote_Time - dt.timedelta(hours = 1)
vix_intraday.Quote_Time = vix_intraday.Quote_Time.map(lambda x: x.replace(second=0))
vix_intraday['Date'] = vix_intraday.Quote_Time.dt.date

In [26]:

vix_intraday.to_csv('vix_intraday.csv')